In [1]:
import numpy.random as rnd
import pandas as pd
from psychopy import visual, data
from psychopy_ext import exp
from collections import OrderedDict
import scripts.computer as computer
import os
import glob

In [5]:
#os.path.splitext(male_full)
#for file in os.listdir("stim/male"):
    #if file.endswith(".JPG"):
       # print(file)
        

ids = {}
male_frontal = [x for x in os.listdir("stim/male") if x.endswith(".JPG")]
female_frontal = [x for x in os.listdir("stim/female") if x.endswith(".JPG")]
ids['male'] = list(set([x.split('_')[1] for x in male_frontal]))
ids['female'] = list(set([x.split('_')[1] for x in female_frontal]))
expressions = ['angry', 'happy', 'neutral']

# remove '(x)' in filenames
rdir = 'stim/female'
def rename(rdir):
    for f in os.listdir(rdir):
        if f.endswith('.JPG') & f.startswith('('):
            os.rename(os.path.join(rdir, f), os.path.join(rdir, f[4:]))

    
def make_trials_ids():
    for c in conditions:
        if c['validity'] == 'valid':
            c['idx_m'] = ids['male'][:]
            rnd.shuffle(ids['male'])
            c['idx_m'] += ids['male'][0:10]
            c['idx_f'] = ids['female'][:]
            rnd.shuffle(ids['female'])
            c['idx_f'] += ids['female'][0:11]
        else:
            rnd.shuffle(ids['male'])
            rnd.shuffle(ids['female'])
            c['idx_m'] = ids['male'][0:10]
            c['idx_f'] = ids['female'][0:10]

In [6]:
print len(ids['male'])
rnd.shuffle(ids['male'])
print len(ids['male'])

20
20


In [171]:
"""
Response Priming Experiment with Emotional Facial Expressions as Stimuli and Response.

This demo is part of psychopy_ext library.
"""

try:
    import pandas
except:
    pass

from psychopy import visual
from psychopy_ext import exp

# some modules are only available in Python 2.6
try:
    from collections import OrderedDict
except:
    from exp import OrderedDict


PATHS = exp.set_paths('mymu', computer)

class MyMu(exp.Experiment):
    """
    Willkommen zu unserem Experiment
    ================================

    Drücken Sie eine **beliebige Taste**, um zu starten.
    """
    def __init__(self,
                 name='mymu',
                 info=OrderedDict([('subjid', 'mm_'),
                                  ('session', 1),
                                  ]),
                 rp=None,
                 actions='run'
                 ):
        super(MyMu, self).__init__(name=name, info=info,
                rp=rp, actions=actions,
                paths=PATHS, computer=computer)

        # user-defined parameters
        self.nvalid = 60
        self.ninvalid = 20
        self.stimsize = (9, 6)  # in deg
        self.isi = 1.2
        
        ids = {}
        male_frontal = [x for x in os.listdir("stim/male") if x.endswith(".JPG")]
        female_frontal = [x for x in os.listdir("stim/female") if x.endswith(".JPG")]
        self.ids['male'] = list(set([x.split('_')[1] for x in male_frontal]))
        self.ids['female'] = list(set([x.split('_')[1] for x in female_frontal]))
        self.factors = {'expression': ['angry', 'happy'], 'validity': ['valid', 'invalid']}
        self.conditions=[
            {'texpr':'anger', 'validity': 'valid', 'pexpr': 'anger', 'weight': self.nvalid},
            {'texpr':'anger', 'validity': 'invalid', 'pexpr': 'happy', 'weight': self.ninvalid},
            {'texpr':'happy', 'validity': 'valid', 'pexpr': 'happy', 'weight': self.nvalid},
            {'texpr':'happy', 'validity': 'invalid', 'pexpr': 'anger', 'weight': self.ninvalid}
            ]
                

    def create_stimuli(self):
        """Define your stimuli here, store them in self.s
        """
        self.create_fixation()
        self.s = {}
        self.s['fix']= self.fixation
        self.s['prime'] = visual.ImageStim(self.win, size=self.stimsize)
        self.s['target'] = visual.ImageStim(self.win, size=self.stimsize)
        self.s['isi'] = visual.ImageStim(self.win, size=(0,0))

    def create_trial(self):
        """Define trial composition
        """
        self.trial = [exp.Event(self,
                                dur=.500,  # in seconds
                                display=self.s['fix'],
                                func=self.idle_event),
                      exp.Event(self,
                                dur=.300,
                                display=self.s['prime'],
                                func=self.idle_event)
                      exp.Event(self,
                                dur=self.isi,
                                display=self.s['isi'],
                                func=self.idle_event)
                      exp.Event(self,
                                dur=2,
                                display=self.s['target'],
                                func=self.idle_event)
                     ]

    def create_exp_plan(self):
        """Put together trials
        DURATION??
        check image files here; randomize here
        """
        # check image stimuli
        if len(ids['female']) != 19 or len(ids['male']) != 20:
                raise Exception('We need 19 unique female and 20 unique male ids')
                
        # create trial list (id, expression, validity)
        # 60 IDs showing Anger in validAnger condition
            # each gender: draw all + draw 10
        # 20 IDs showing Anger in invalidHappy condition
            # # each gender: draw 10
        # 60 IDs showing Happy in validHappy condition
            # each gender: draw all + draw 10
        # 20 IDs showing Happy in invalidAnger condition
            # # each gender: draw 10
            
        # Wie genau den Trialhandler nutzen??? (wenn dann data.TrialHandlerExt)
        # wohl besser: loop über valide und invalide conditions getrennt
        #
        # jeden faktor durchgehen, jeweils 160 item-lange list erzeugen, auf reihenfolge achten!
        # an_val (60), an_inval(20), ha_val(60), ha_inval(20):
        #
        # texpr_an (80), texpr_ha(80)
        # val (60), inval(20), val(60), inval(20)
        # pexpr_an (60), pexpr_ha(20), pexpr_ha(60), p_expr_an(20)
        # 30 idx_m, 30 idx_f, 10 idx_m, 10 idx_f, 30 idx_x, 30 idx_f, 10 idx_m, 10 idx_f
        # 30 male, 30 female, 10 male, 10 female, 30 male, 30 female, 10 male, 10 female
        # 
            
        conditions = data.createFactorialTrialList(self.factors)
        
        exp_plan = []
        for trialno in range(self.ntrials):
            exp_plan.append(OrderedDict([
                        ('trialno', trialno),
                        ('texpr', expr),
                        ('validity', val),
                        ('pexpr', pexpr),
                        ('sex', sex),
                        ('id', idx),
                        ('onset', ''),  # empty ones will be filled up
                        ('dur', ''),    # during runtime
                        ('corr_resp', 1),
                        ('subj_resp', ''),
                        ('accuracy', ''),
                        ('rt', ''),
                        ]))
        self.exp_plan = exp_plan
        
        
    def before_trial(self):
        """Set up stimuli prior to a trial
        """
        im_fname = os.path.join(self.paths['images'], self.this_trial['im'])
        self.s['bitmap1'].setImage(im_fname + self.asfx)
        self.s['bitmap1'].setOri(self.this_trial['ori'])
        self.s['bitmap2'].setImage(im_fname + self.bsfx)
        self.s['bitmap2'].setOri(self.this_trial['ori'])
        self.bitmap = self.s['bitmap1']

        if self.thisTrialN > 0:  # no need for instructions for the first trial
            self.show_text(text=self.trial_instr, wait=0)

        

SyntaxError: invalid syntax (<ipython-input-171-ab065514ba1b>, line 84)

In [8]:
MyMu(rp={'no_output':True, 'debug':True}).run()

trial 1

c:\users\ddmitarbeiter\anaconda2\lib\site-packages\numpy\ma\core.py:3248: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  _data[indx] = dval
c:\users\ddmitarbeiter\anaconda2\lib\site-packages\numpy\ma\core.py:3249: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  _mask[indx] = mval


SystemExit: 0

c:\users\ddmitarbeiter\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [51]:
from psychopy import visual, core

win = visual.Window(size=(600,400))
bitmap = visual.ImageStim(win, size=(0,0))
bitmap.draw()  # draw bitmap on the window
win.flip()  # make bitmap visible
core.wait(3)  # seconds
win.close()